# **Notebook de test de Téléchargement de donnée de basket depuis le site nba.com**

Sommaire

## Imports

In [1]:
%load_ext autoreload
%autoreload 2
from TelechargementDonnees import CreationDriverFirefox, JourneeSiteNba, PasDeMatchError, JoueursSiteNba
import pandas as pd
import numpy as np

## 1. Telecharger les données liees aux matchs depuis le site internet pour une ou plusieurs dates  
1. on telecharge une journee
1. on sauvegarder dans un json

In [ ]:
#telecharger une journee
try : 
    journee=JourneeSiteNba('2021-01-02')
except PasDeMatchError as e : 
    print(e)
#exporter
#journee.saveCsv()

In [4]:
# pour un ensemeble de journee : 
for j in ['2021-01-%02d' %n  for n in range(6,8)] : 
    print(j)
    try :
        journee=JourneeSiteNba(j)
        journee.saveCsv()
    except PasDeMatchError as e: 
        print(e)    
        continue

2021-01-06
0 https://fr.global.nba.com/boxscore/#!/0022000107
1 https://fr.global.nba.com/boxscore/#!/0022000108
2 https://fr.global.nba.com/boxscore/#!/0022000109
3 https://fr.global.nba.com/boxscore/#!/0022000110
4 https://fr.global.nba.com/boxscore/#!/0022000111
5 https://fr.global.nba.com/boxscore/#!/0022000112
6 https://fr.global.nba.com/boxscore/#!/0022000113
7 https://fr.global.nba.com/boxscore/#!/0022000114
8 https://fr.global.nba.com/boxscore/#!/0022000115
9 https://fr.global.nba.com/boxscore/#!/0022000116
10 https://fr.global.nba.com/boxscore/#!/0022000117
2021-01-07
0 https://fr.global.nba.com/boxscore/#!/0022000118
1 https://fr.global.nba.com/boxscore/#!/0022000119
2 https://fr.global.nba.com/boxscore/#!/0022000120
3 https://fr.global.nba.com/boxscore/#!/0022000121
4 https://fr.global.nba.com/boxscore/#!/0022000122


## 2. Telecharger les données liees aux joueurs depuis le site internet 
1. on se connecte à la page principale des joueurs
1. on passe de lettre en lettre pour sauvegarder les donnees

In [62]:
#creation de la classe
pagejoueuer=JoueursSiteNba()
#acces aux donnees
pagejoueuer.dfJoueurs
#export des donnees
pagejoueuer.dfJoueurs.reset_index(drop=True).to_json(r'C:\Users\martin.schoreisz\Documents\AffairesEnCours\temp\basket\Joueurs_saison_2020-2021\Joueurs_saison_2020-2021.json')